# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('project_1')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [17]:
query = """CREATE TABLE IF NOT EXISTS music_app_history (
        sessionid INT,
        iteminsession INT,
        userid INT,
        artist TEXT,
        songtitle TEXT,
        firstname TEXT,
        lastname TEXT,
        length DOUBLE,
        gender TEXT,
        level TEXT,
        location TEXT,
        PRIMARY KEY ((sessionid, iteminsession))
    ) """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = """CREATE TABLE IF NOT EXISTS songsbyusersession (
            userid INT,
            sessionid INT,
            iteminsession INT,
            artist TEXT,
            songtitle TEXT,
            firstname TEXT,
            lastname TEXT,
            length DOUBLE,
            gender TEXT,
            level TEXT,
            location TEXT,
            PRIMARY KEY ((userid, sessionid), iteminsession)
        )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = """CREATE TABLE IF NOT EXISTS usersbysong (
            songtitle TEXT,
            userid INT,
            firstname TEXT,
            lastname TEXT,
            PRIMARY KEY (songtitle, userid)
        )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

### Insert data into the tables

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionid, iteminsession, userid, artist, songtitle, firstname, lastname, length, gender, level, location) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (
        int(line[8]), int(line[3]), int(line[10]),
        line[0], line[9], line[1], line[4],
        float(line[5]), line[2], line[6],
        line[7]
        )

        session.execute(query, values)

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songsbyusersession (sessionid, iteminsession, userid, artist, songtitle, firstname, lastname, length, gender, level, location) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (
        int(line[10]), int(line[8]), int(line[3]),
        line[0], line[9], line[1], line[4],
        float(line[5]), line[2], line[6],
        line[7]
        )

        session.execute(query, values)

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO usersbysong (songtitle, userid, firstname, lastname) "
        query += "VALUES (%s, %s, %s, %s)"
        values = (
        line[9], int(line[10]), line[1], line[4]
        )

        session.execute(query, values)

#### Do a SELECT to verify that the data have been inserted into each table

In [23]:
query = "SELECT artist, songtitle, length FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.songtitle, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [34]:
query = "SELECT * FROM songsbyusersession WHERE userid > 10 AND sessionid > 1 LIMIT 10 ALLOW FILTERING"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.songtitle, row.firstname, row.lastname)

                    

Muse Supermassive Black Hole (Twilight Soundtrack Version) Jacqueline Lynch
Enrique Iglesias Ring My Bells Harper Barrett
Riverside Stuck Between Harper Barrett
Blitzen Trapper Murder Babe (Album) Harper Barrett
The Foreign Exchange All That You Are (Instrumental) Layla Griffin
Primus Over The Electric Grapevine Layla Griffin
Steppenwolf Born To Be Wild Sara Johnson
Wet Wet Wet Goodnight Girl Sara Johnson
Faith No More Last Cup Of Sorrow Hayden Brock
Sara Bareilles Gravity Hayden Brock


In [35]:
query = "SELECT firstname, lastname FROM usersbysong WHERE songtitle = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)
                   

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Answering the questions

#### 1. Question: Return artist, song title, and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

- Description: This query retrieves information about the artist, song title, and length of a song played during a specific session and item in the music app history.
- Partition Key: sessionid - To efficiently group data by session.
- Clustering Key: iteminsession - To uniquely identify items (songs) within a session.

In [36]:
from prettytable import PrettyTable

# Execute query
query = "SELECT artist, songtitle, length FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    exit()

# Print output with PrettyTable
table = PrettyTable()

table.field_names = ["Artist", "Song Title", "Length"]

for row in rows:
    table.add_row([row.artist, row.songtitle, row.length])

table.align["Artist"] = "l"
table.align["Song Title"] = "l"
table.align["Length"] = "l"

print(table)


+-----------+---------------------------------+----------+
| Artist    | Song Title                      | Length   |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


Answer: Only one song (see table above) matches the criteria in the where clause of the query.

#### 2. Question: Returnthe following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- Description: This query retrieves information about the artist, song title, first name, and last name for a specific user and session item from the songs played during user sessions.
- Partition Key: (userid, sessionid) - To efficiently group data by user and session.
- Clustering Key: iteminsession - To uniquely identify items (songs) within a session for a user.

In [37]:
from prettytable import PrettyTable

# Execute query
query = "SELECT artist, songtitle, firstname, lastname FROM songsbyusersession WHERE userid = 10 AND sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    exit()

# Print output with PrettyTable
table = PrettyTable()

table.field_names = ["Artist", "Song Title", "First Name", "Last Name"]

for row in rows:
    table.add_row([row.artist, row.songtitle, row.firstname, row.lastname])

table.align["Artist"] = "l"
table.align["Song Title"] = "l"
table.align["First Name"] = "l"
table.align["Last Name"] = "l"

print(table)


+--------+------------+------------+-----------+
| Artist | Song Title | First Name | Last Name |
+--------+------------+------------+-----------+
+--------+------------+------------+-----------+


Answer: No song matches the criteria in the where clause.

#### 3. Question: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- Description: This query retrieves the first name and last name of users who played a specific song, 'All Hands Against His Own.'
- Partition Key: songtitle - To efficiently group data by song.
- Clustering Key: userid - To uniquely identify users for a specific song.

In [38]:
from prettytable import PrettyTable

# Execute query
query = "SELECT firstname, lastname FROM usersbysong WHERE songtitle = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    exit()

# Print output with PrettyTable
table = PrettyTable()

table.field_names = ["First Name", "Last Name"]

for row in rows:
    table.add_row([row.firstname, row.lastname])

table.align["First Name"] = "l"
table.align["Last Name"] = "l"

print(table)


+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline | Lynch     |
| Tegan      | Levine    |
| Sara       | Johnson   |
+------------+-----------+


Answer: The three users above listened to this song.

### Drop the tables before closing out the sessions

In [21]:
query = "DROP table music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP table songsbyusersession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

query = "DROP table usersbysong"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()